# End of week 1 exercise

OpenAI APIとOllamaに慣れていることを証明するために、技術的な質問を受け取って説明で回答するツールを作ってね。これはコース中に君自身が使えるツールになるよ！

In [6]:
# imports

import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [7]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [27]:
# set open ai APi key

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")

# set up environment

openai = OpenAI()
OLLAMA_API = "http://localhost:11434/api/chat"
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

API key looks good so far


In [25]:
# set up llama

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [10]:
# system prompt and user prompt

system_prompt = "あなたはプロの開発者です。質問に対して日本語で回答してください。"

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [28]:
def create_answer(model, ai):
    stream = ai.chat.completions.create(
        model = model,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question},
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [29]:
# Get gpt-4o-mini to answer, with streaming

create_answer(MODEL_GPT, openai)

このコードは、Pythonにおける「ジェネレーター」を使用した構文で、特定の条件を満たす要素から新しいイテラブルを生成する目的で使用されています。以下に、コードの各部分を分解して説明します。

### コードの説明

1. **set comprehension** (`{... for book in books if book.get("author")}`):
   - `books`というリスト（または他のイテラブル）から要素を一つずつ取り出し、各要素を`book`という変数に格納します。
   - `book.get("author")`が真（存在する場合）である場合に限り、その`author`の値を集めて集合（`set`）を作成します。
   - この集合は、同じ著者が複数の本に記載されている場合でも一意の著者のみを含みます。

2. **yield from**:
   - `yield from`は、特定のイテラブルオブジェクトから値を生成するための構文です。この場合、先ほど作成した集合の要素を一つずつ生成（`yield`）します。
   - つまり、`books`の中で著者が存在する本の著者名を一つずつ返すジェネレーターとなります。

### なぜこのコードが使用されるのか

- **効率性**: `yield from`を使用することで、全ての著者を一度にリストとして返すのではなく、必要に応じて一つずつ返すことができ、メモリの使用量が少なくて済みます。
- **一意性**: セットを用いることで、重複する著者名を自動的に排除します。
- **可読性**: コードが簡潔で、何を意図しているかが分かりやすいです。

このコードは、主に本のリストから著者を集めた際に、一意の著者名を効率よく取り出したい場合に役立ちます。

In [30]:
# Get Llama 3.2 to answer

create_answer(MODEL_LLAMA, ollama_via_openai)

このコーディングは、特定の属性となる書籍を取得するために使用されている。これには以下の点で理解できる。

1. `yield from`: このキーワードは、生成子 (generator) を other への transmission に使われる。つまり、一度生成された結果を受け取ること。
2. `itertools.chain`: この要因は一般的に、複数の可変量源を一つの可変量に結合するために使用される。但し、このコードではその名が言及されていない。
3.`book.get("author")`: この部分では書籍の各要素に対して、
   - `book`: つまり、生成される可能性のある各要素。これは、一度生成された元となるデータセット。
   - `get()`: 指定するカレントに存在する値が特定されれば、その値が返るメソッドである。これは、key-less dictionary の使用を示唆しているが、実際は `book` も key-less である可能性がある。

つまり、このコードは次の機能を提供します。
1. すべての書籍から "author" にある値が含まれているものを取得する必要があります。
2. 複数のデータソースを処理し、特定の値が存在する要素を取り出すときに使用されるため、yield from と chains の概念に対応します。
3. `for book in books if book.get("author")` は、`book.get("author")` が値 "author" を持つすべての book から data を取得します。

これで理解していることを確認してください。